# Planar Straight-Line Graph & Doubly-Connected Edge List

## Введение

#### Откуда возникает задача (карты по слоям)

Для того, чтобы нам лучше понимать, о чём же сейчас пойдёт речь, давайте представим себе карты местности. Какие вообще бывают карты местности? Совершенно любые. На них могут быть отмечены реки, железнодорожные маршруты, точки с названиями населённых пунктов, болота какие-нибудь. И иногда это бывает проблемой. Если мы захотели найти на карте нужный нам населённый пункт, то нанесённые на карту названия рек будут нас только отвлекать. Поэтому, чтобы такой проблемы не возникало, в географических системах карты делят на тематические слои. Например, один слой содержит только дороги. А другой слой только реки. И так далее. Каждый слой содержит только один тип информации. При этом, по смыслу эта информация может быть совершенно любой. Слои могут обозначать какие-то специфические зоны на карте. С геометрической точки зрения, слои могут быть разными. Скажем, слой с дорогами представляет собой множество отрезков. А слой с городами будет простым множеством точек. А если взять, например, районы, то они будут представлять собой разбиение карты на области. 

Люди, которые пользуются такой системой карт со слоями, могут довольно легко найти то, что им нужно. Если человеку нужно найти маршрут до города, то сначала он открывает слой с городами, а потом, когда он нашёл город, он открывает слой с дорогами и ищет маршрут до города. Современные системы такого типа позволяют ещё и брать пересечения нескольких слоёв. Допустим, что у нас есть слой дорог и слой рек. Тогда пересечение этих слоёв будет представлять собой множество мостов. 

#### Задача, которая разбирается в этом конспекте

Как было сказано, при пересечении слоёв как раз и возникает задача о том, как их правильно пересечь. Слои могут быть разными с геометрической точки зрения. Самый очевидный пример - это когда оба слоя представляют собой множества отрезков. Этот случай разобран в другом конспекте и, следовательно, здесь его не будет. Нас интересует более общий случай, когда слои представляют собой разбиения карты на области. Для того, чтобы делать какие-то алгоритмы с этими областями, их сначала нужно представить в виде удобной структуры. Такой структурой как раз и является DCEL. 

## Определения (неформальные) и пояснения к ним

**Разбиение плоскости** - разделение плоскости на области. Для примера можно посмотреть на диаграмму Вороного.

**Планарный граф** - граф, который можно так нарисовать на плоскости, чтобы его рёбра не пересекались. 

**Планарный прямолинейный граф (ППЛГ, PSLG)** - планарный граф с прямыми рёбрами. 

**Разбиение плоскости, порождённое ППЛГ** - если мы нарисуем наш граф на плоскости, то он разделит её на области. Таким образом, у нас получится разбиение плоскости. Если вернуться к истории про карты, то слой в карте, который делит карту на области, как раз может быть представлен в виде планарного прямолинейного графа. 

**Область (face)** - Если мы нарисуем наш граф на плоскости и посмотрим на него, то face - это множество точек, которое ограничено по периметру рёбрами графа (здесь будет иллюстрация из Де Берга).

## Какие операции позволяет делать DCEL 

Итак, мы сказали, что разбиение плоскости может быть порождено планарным прямолинейным графом. То есть, сам граф, по сути, уже является структурой, которая может представлять слой в карте. Но у такой структуры нет никаких методов и пользы от неё будет не очень много. А каких методов нам бы хотелось? Например, по заданной точке узнать, к какой области она принадлежит. Но это - довольно сложная задача (которая, впрочем, также разобрана в другом конспекте про локализацию). DCEL является более базовым представлением. Впрочем, он позволяет делать следующее:
1. Обойти по периметру заданную область
2. Получать смежные области, если нам дали общее ребро.
3. Посетить все рёбра вокруг заданной точки

## Из чего состоит DCEL

Теперь давайте разберёмся с тем, что же всё-таки DCEL представляет из себя как структура данных. Если в двух словах, то он хранит в себе 3 набора данных: данные о вершинах, данные о рёбрах и данные об областях. При этом, кроме данных, которые необходимы для того, чтобы хранить геометрию, есть ещё аттрибуты. Например, мы разбили карту на области и каждая область представляет собой, скажем, страны. Тогда аттрибутом каждой области будет название страны, которая занимает данную область. 

Теперь распишем формально, что хранится для вершин, рёбер и областей

#### вершины

1. Координаты 
2. Указатель на произвольное инцидентное полуребро (о полурёбрах далее)

#### рёбра (полурёбра)

Для того, чтобы мы могли ходить по периметру области, ребро должно хранить указатель на следующее ребро. Но так как обычно ребро  граничит с двумя областями, нам понадобится сразу 2 указателя. Поэтому мы разбиваем ребро на 2 полуребра. Полуребро уже будет иметь ровно одно следующее (и предыдущее) полуребро при обходе. Каждое полуребро хранит указатель на полуребро-близнец. Полурёбра ориентированы таким образом, что если наблюдатель будет двигаться вдоль по направлению полуребра, область этого полуребра будет находиться слева от наблюдателя. Из этого следует, что в одном ребре полурёбра будут ориентированы в разные стороны относительно друг друга. Итак, пусть e - это полуребро. Для него хранится следующее:

1. Origin(e) - точка начала ребра
2. Twin(e) - второе полуребро из исходного ребра
3. IncidentFace(e) - указатель на область, которую мы обходим
4. Next(e) - следующее полуребро в обходе
5. Prev(e) - предыдущее ребро в обходе

#### области (faces)

1. OuterComponent(f) - указатель на некоторое полуребро из внешней границы. Если область не ограничена, то null.
2. InnerComponents(f) - список указателей на полурёбра дырок в данной области. По одному указателю на дырку. Про дырки ещё ничего не было сказано. Они отличаются тем, что для них направление обхода не против часовой стрелки, а по часовой. Это нужно для того, чтобы для них область тоже всегда была слева относительно направления вдоль ребра.   
